In [1]:
# Import the necessary libraries

import re
from numpy import array
from numpy import asarray
from numpy import zeros

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.regularizers import L2
from keras.layers.embeddings import Embedding
from keras.layers import Dense, merge, Dropout, Input
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from config import *

[nltk_data] Downloading package punkt to
[nltk_data]     /home/aakashj2412/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Helper function that cleans the input data and enumerates the labels

def extract(s):
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

labels = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

In [5]:
# Function that reads data and parses data from file

def readFileData(filePath):
    with open(filePath, 'r') as f:
        inputRows = [row.split('\t') for row in f.readlines()[1:]]

    inputPremises = [extract(row[1]) for row in inputRows if row[0] in labels]
    inputHypotheses = [extract(row[2]) for row in inputRows if row[0] in labels]
    inputLabels = [labels[row[0]] for row in inputRows if row[0] in labels]
    f.close()

    return [inputPremises, inputHypotheses, inputLabels]

In [6]:
# Reading train and test data

trainData = readFileData(trainFilePath)
testData = readFileData(testFilePath)

In [7]:
# Hyperparameters

maxLen = 50
epochs = 10
batchSize = 256
gloveDimension = 100
regularization = 4e-6

In [8]:
# Tokenizer to generate the vocabulary of the system

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainData[0] + trainData[1])
vocabSize = len(tokenizer.word_index)+1

In [9]:
# Convert the train data to sequences as per the vocabulary
trainData[0] = tokenizer.texts_to_sequences(trainData[0])
trainData[1] = tokenizer.texts_to_sequences(trainData[1])

# Pad or trim all generated sequences to the same max sentence length
trainData[0] = pad_sequences(trainData[0], maxLen, padding='post')
trainData[1] = pad_sequences(trainData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
trainData[2] = tf.keras.utils.to_categorical(trainData[2], num_classes=3)

In [10]:
# Convert the test data to sequences as per the vocabulary
testData[0] = tokenizer.texts_to_sequences(testData[0])
testData[1] = tokenizer.texts_to_sequences(testData[1])

# Pad or trim all generated sequences to the same max sentence length
testData[0] = pad_sequences(testData[0], maxLen, padding='post')
testData[1] = pad_sequences(testData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
testData[2] = tf.keras.utils.to_categorical(testData[2], num_classes=3)

In [11]:
# Load the GloVe file, via config
embeddingsDict = dict()
glove = open(gloveVectorsPath)

# Iterate through the GloVe file to store all the embeddings in a dictionary
for line in glove:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddingsDict[word] = vector_dimensions

glove.close()

In [12]:
# Iterate through the embeddings and store only those that are present in our vocabulary
embeddingsMat = zeros((vocabSize, 100))
for word, index in tokenizer.word_index.items():
    vec = embeddingsDict.get(word)
    if vec is not None:
        embeddingsMat[index] = vec

In [14]:
# Define the embedding layer for our baseline RNN model
embed = Embedding(vocabSize, gloveDimension, weights=[embeddingsMat], input_length=maxLen, trainable=False)

# As Premise and Hypothesis are distinct and are to be inputted separately, define two inputs and embed
premise = Input(shape=(maxLen,), dtype='int32')
hypothesis = Input(shape=(maxLen,), dtype='int32')

premInput = embed(premise)
hypoInput = embed(hypothesis)

In [15]:
# Once the sentence embeddings have been generated, generate a matrix of dimensions maxLen X gloveDimension
# On adding maxLen, we get a single embedding vector of length gloveDimension

rnn = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=gloveDimension)

In [16]:
# Apply batch normalization to the two input embeddings separately

premInput = rnn(premInput)
hypoInput = rnn(hypoInput)
premInput = tf.keras.layers.BatchNormalization()(premInput)
hypoInput = tf.keras.layers.BatchNormalization()(hypoInput)

In [17]:
# Joint is a concatenated embeddings layer, generated from the premise and hypothesis inputs
# Dilution of probability 0.2, to assist in regularization
joint = keras.layers.concatenate([premInput, hypoInput])
joint = Dropout(0.2)(joint)
for i in range(3):
    joint = Dense(200, activation='tanh', kernel_regularizer=L2(regularization))(joint)
    joint = Dropout(0.2)(joint)
    joint = tf.keras.layers.BatchNormalization()(joint)

# 3 layers of the TanH activation function, along with L2 regularization.
# The final decision is based on the Softmax function
pred = Dense(3, activation='softmax')(joint)

In [18]:
# Defining the final models input and output format, as well as compilation parameters

model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# Fitting the model using the train data

model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs)

Epoch 1/10
2146/2146 [==============================] - 17s 7ms/step - loss: 0.9950 - accuracy: 0.5153
Epoch 2/10
2146/2146 [==============================] - 18s 8ms/step - loss: 0.9243 - accuracy: 0.5666
Epoch 3/10
2146/2146 [==============================] - 19s 9ms/step - loss: 0.9004 - accuracy: 0.5838
Epoch 4/10
2146/2146 [==============================] - 19s 9ms/step - loss: 0.8872 - accuracy: 0.5930
Epoch 5/10
2146/2146 [==============================] - 19s 9ms/step - loss: 0.8794 - accuracy: 0.5985
Epoch 6/10
2146/2146 [==============================] - 19s 9ms/step - loss: 0.8742 - accuracy: 0.6027
Epoch 7/10
2146/2146 [==============================] - 19s 9ms/step - loss: 0.8688 - accuracy: 0.6055
Epoch 8/10
2146/2146 [==============================] - 19s 9ms/step - loss: 0.8658 - accuracy: 0.6082
Epoch 9/10
2146/2146 [==============================] - 19s 9ms/step - loss: 0.8629 - accuracy: 0.6110
Epoch 10/10
2146/2146 [==============================] - 20s 9ms/step - l

In [20]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

39/39 [==============================] - 0s 4ms/step - loss: 0.7860 - accuracy: 0.6607
Loss =  0.7859595417976379
Acc =  0.6607288122177124
